In [0]:
import os
import zipfile

local_zip = '/tmp/text region detectoin data set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/text region detectoin data set')

In [0]:
# Directory with our text region pictures
train_text_dir = os.path.join('/tmp/text region detectoin data set/text region')

# Directory with our non text region pictures
train_non_text_dir = os.path.join('/tmp/text region detectoin data set/non text region')

In [11]:
train_text_names = os.listdir(train_text_dir)
print(train_text_names[:10])

train_non_text_names = os.listdir(train_non_text_dir)
print(train_non_text_names[:10])

print('total training text images:', len(os.listdir(train_text_dir)))
print('total training non text images:', len(os.listdir(train_non_text_dir)))

['gray scale modified 1280.jpg', 'gray scale modified 447.jpg', 'gray scale modified 250.jpg', 'gray scale modified 2263.jpg', 'gray scale modified 1149.jpg', 'gray scale modified 1921.jpg', 'gray scale modified 1203.jpg', 'gray scale modified 3039.jpg', 'gray scale modified 979.jpg', 'gray scale modified 431.jpg']
['gray scale modified 1280.jpg', 'gray scale modified 447.jpg', 'gray scale modified 250.jpg', 'gray scale modified 2263.jpg', 'gray scale modified 1149.jpg', 'gray scale modified 1921.jpg', 'gray scale modified 1203.jpg', 'gray scale modified 979.jpg', 'gray scale modified 431.jpg', 'gray scale modified 2323.jpg']
total training text images: 3154
total training non text images: 2688


In [0]:
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 224x224 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(filters = 96, kernel_size = (3, 3), strides = (4, 4), padding = 'same', activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    # The second convolution
    tf.keras.layers.Conv2D(filters = 256, kernel_size = (11, 11), strides = (1, 1), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    # The third convolution
    tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    # The fourth convolution
    tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    # The fifth convolution
    tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    # Flatten the results to feed into a Dense Layer
    tf.keras.layers.Flatten(),
    # The first 4096 Dense layer
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.BatchNormalization(),

    # The second 4096 Dense layer
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.BatchNormalization(),

    # The second 1000 Dense layer
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.BatchNormalization(),

    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 56, 56, 96)        2688      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 96)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 28, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 256)       2973952   
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 14, 14, 256)       1024      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 384)      

In [0]:
from tensorflow.keras.optimizers import RMSprop

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/text region detectoin data set/',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=128,
        class_mode='binary')

Found 5840 images belonging to 2 classes.


In [23]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15)

Epoch 1/15
8/8 [==============================] - 266s 33s/step - loss: 0.4769 - accuracy: 0.8096
Epoch 2/15
8/8 [==============================] - 261s 33s/step - loss: 0.4106 - accuracy: 0.8418
Epoch 3/15
8/8 [==============================] - 267s 33s/step - loss: 0.3952 - accuracy: 0.8477
Epoch 4/15
8/8 [==============================] - 261s 33s/step - loss: 0.4771 - accuracy: 0.8125
Epoch 5/15
8/8 [==============================] - 263s 33s/step - loss: 0.3904 - accuracy: 0.8418
Epoch 6/15
8/8 [==============================] - 244s 31s/step - loss: 0.3525 - accuracy: 0.8473
Epoch 7/15
8/8 [==============================] - 263s 33s/step - loss: 0.3631 - accuracy: 0.8623
Epoch 8/15
8/8 [==============================] - 259s 32s/step - loss: 0.3507 - accuracy: 0.8604
Epoch 9/15
8/8 [==============================] - 263s 33s/step - loss: 0.3543 - accuracy: 0.8438
Epoch 10/15
8/8 [==============================] - 258s 32s/step - loss: 0.2866 - accuracy: 0.8750
Epoch 11/15
8/8 [==